# Harrow-Hassidim-Lloyd (HHL) algorithm 
## to solve linear system of equations


<br />

**References:**

*1. arXiv: https://arxiv.org/abs/1703.06613* <br />
*2. arXiv: https://arxiv.org/abs/1302.1946* <br />
*3. arXiv: https://arxiv.org/abs/1804.03719* <br />
*4. arXiv: https://arxiv.org/abs/1802.08227* <br />

#### What?

Given $A\vec x = \vec b$ for a Matrix $A$ and vector $\vec b$, solve for $\vec x$.

#### Conditions:

$A$ is Hermitian matrix i.e. $A = A^\dagger$. $A$ must be sparse and low-rank.

We have to represent $\vec x$ and $\vec b$ as quantum states, $|x\rangle$ and $|b\rangle$. So, we normalize them, i.e. $|x| = |b| = 1$

#### Now what?

We have to solve $A|x\rangle = |b\rangle$ for $|x\rangle$.

So, we can rewrite, $|x\rangle = \frac{A^{-1}|b\rangle}{|A^{-1}|b\rangle|}$

### How to do it?

We have to find $\vec x$. But instead of finiding $\vec x$ directly, the question is posed as following:

Find the expectation value of some operator $M$ with $\vec x$ i.e. $\vec x M \vec x$. 

The operator here is typically the Pauli operators X,Y,Z. The quantum computer returns us the answer in probabilities of measurement which can be translated to get expectation values with respect to operators.

#### Details:

Let {$|u_{j}\rangle$} be the eigenbasis of $A$.

Let {$\lambda_j$} be the eigenvalues of $A$. Scale the eigenvalues such that, 0 < $\lambda_j$ < 1.

State $|b\rangle$ can be written as the linear combination of eigenbasis {$|u_j\rangle$}, as follows: 

$|b\rangle$= $\sum_{j=1}^N \beta_j\frac{1}{\lambda_j}|u_j\rangle$


$A$ can be written as:  $A = R^\dagger\Lambda R$ ----- <span style="color:red"> What is the R operator?</span>

Now, HHL algorithm involves the following 3 steps (Ref: [1]):

$$R^\dagger\Lambda R|x\rangle = |b\rangle \overset{Step 1}{\Rightarrow} \Lambda R|x\rangle = R|b\rangle \overset{Step 2}{\Rightarrow} R|x\rangle = \Lambda^{-1} R|x\rangle \overset{Step 3}{\Rightarrow} |x\rangle = R^\dagger \Lambda^{-1} R |x\rangle$$ 

### Qubits Required:

 - *1* ancilla qubit <br />
 - *n* qubits to store eigen values of A in binary format --- with precision up to *n* bits. <br />
 - a memory of *O(*log*(N))*, that initially stores $|b\rangle$ and later $|x\rangle$.

Any quantum algorithm, proceeds as follows:<br />
Initialization $\longrightarrow$ Superposition $\longrightarrow$ Unitary Transformation $\longrightarrow$ Post Processing $\longrightarrow$ Measurement

### HHL Algorithm:

1. **Initialization** : $|0\rangle_a, |0\rangle_r, |b\rangle_m$ = for *ancilla*, *register* and *memory* <br />
 Initial state, therefore is: $|0\rangle_a |0\rangle_r |b\rangle_m$<br />
 
2. <span style="color:blue">**(Step 1)**</span> Perform quantum phase estimation using the unitary transformation $e^{iAt}$, where *t* is the time. <br />

3. Map the eigenvalues $\lambda_j$ into the register (in binary form), so the state: $|0\rangle_a |0\rangle_r |b\rangle_m$ becomes:
$$|0\rangle_a |0\rangle_r |b\rangle_m \Rightarrow \sum_{j=1}^N \beta_j |0\rangle_a |\lambda_j\rangle_r |u_j\rangle_m$$
<br />

4. <span style="color:blue">**(Step 2)**</span> Rotate the ancilla qubit $|0\rangle_a$ to $\sqrt{1 - \frac{C^2}{\lambda_{j}^2}}|0\rangle_a + \frac{C}{\lambda_j}|1\rangle_a$ for each $\lambda_j$. This is done by controlled rotation on ancilla qubit $|0\rangle_a$.

5. After Step 2, the state becomes:
$$\sum_{j=1}^N \beta_j \Biggl(\sqrt{1-\frac{C^2}{\lambda_{j}^2}}|0\rangle_a + \frac{C}{\lambda_j}|1\rangle_a\Biggl)|\lambda_j\rangle_r|u_j\rangle_m$$

6. <span style="color:blue">**(Step 3)**</span> Uncompute. Reverse Step 1, the state we get is:
$$\sum_{j=1}^N \beta_j \Biggl(\sqrt{1-\frac{C^2}{\lambda_{j}^2}}|0\rangle_a + \frac{C}{\lambda_j}|1\rangle_a\Biggl)|0\rangle_r|u_j\rangle_m$$
This step does inverse phase estimation and disentanglement. This restores the register to $|0\rangle$.

7. **Measurement** : By selecting the $|1\rangle$ in ancilla qubit, we get the following on the memory qubit:<br />
In the memory qubit, we get: $|x\rangle \approx \sum_{j=1}^N C \Biggl ( \frac{\beta_j}{\lambda_j}\Biggl ) |u_j\rangle$

------------

### Example: Given, 

$A = \frac{1}{2} \Biggl [\begin{matrix} 3 & 1 \\ 1 & 3 \end{matrix}\Biggl ]$ 

$b = \Biggl( \begin{matrix} 1 \\ 0 \end{matrix}\Biggl )$, $|-\rangle = \frac{1}{\sqrt2}\Biggl( \begin{matrix}1 \\ -1 \end{matrix}\Biggl )$, $|+\rangle = \frac{1}{\sqrt2}\Biggl( \begin{matrix} 1 \\ 1 \end{matrix}\Biggl )$,

eigen values: 

$\lambda_1 = 1$, so $\theta = \pi$;<br />
$\lambda_2 = 2$, so $\theta = \pi/3$  ----- Done by setting C = 1 in equation of <span style="color:blue"> **(Step 2)**</span>

$\lambda = \phi$ = 0 in controlled unitary rotation. ----- <span style="color:blue"> Use cu3 rotation from QISKit. </span> <span style="color:red"> Also, how's that helping?? </span>

#### The circuit for the above algorithm will look as follows:

![hhl-circuit](images/hhl-example1.png)

### Calculations:

The controlled rotations are done as follows:

---- TO DO ----


### The QISKit Program for HHL is done as follows:

In [34]:
## ---- Incomplete ----
## BTW... TO RUN THIS, YOU NEED QISKit installed on your system. 
## Use Anaconda and create a separate environment for installing QISKit.

## Preliminary implementation for 2 x 2 matrix. Refer, arXiv: 1703.06613
## Next: Implement for arbitrary 2 x 2, 3 x 3, 5 x 5...

## Initialize
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute
from qiskit import BasicAer as Aer

## Declare Classical and Quantum Registers and Create the Quantum Circuit Class
q = QuantumRegister(6)
c = ClassicalRegister(2)
hhl = QuantumCircuit(q, c)

## Quantum Phase Estimation

# Superposition     ### some thing is probably wrong here.
hhl.h(q[3])
# Controlled-U0
hhl.cx(q[2], q[3])
# Controlled-U1
hhl.cx(q[1], q[2])

hhl.x(q[2])

## Swapping Operation
hhl.swap(q[1], q[2])

## Conditional Rotation of ancilla Qubit
hhl.cu3(3.1415926535, 0, 0, q[2], q[0]);
hhl.cu3(1.0471975512, 0, 0, q[1], q[0]);

## Uncompute Circuit ### Make updates as per changes in step 1.
hhl.swap(q[1], q[2])
hhl.x(q[2])
hhl.cx(q[1], q[2])
hhl.cx(q[2], q[3])
hhl.h(q[3])

## Something is missing here.

## Measurement
hhl.measure(q[0], c[0])
hhl.measure(q[3], c[1])



## Run Circuit
backend = Aer.get_backend('qasm_simulator')
job = execute(hhl, backend, shots=100)
result = job.result()
counts = result.get_counts(hhl)
hhl.draw();
print(counts);

{'01': 29, '00': 71}


### Important Challenges:
1. How to obtain/calculate the expectation values? Measure probability of states in say, 4096 shots? Plot.
2. Quantum Phase Estimation - Amplitude Amplification - QRAM --- how, How???!
3. How to infer result from the measurement result + effect of X, Y, Z measurement on ancilla.
4. Effect of C (clock) --- ticking in the register?
5. What about $t_0$ in the matrix A?
6. Explore the idea of Hamiltonian encoding of A to scale this.
7. 


-----
Interesting Ideas:
- Change of basis for quantum measurement.
- Talk to Makena.
- QITE and QLanczos --- wavefunction is encoded in the quantum circuit. Better than VQEs.
- QGANs to generate track data?
- Quantum Random Walks (what's that?)


----
More Questions:
- 5 parameters of track. What’s the range? <br />
Encode in diagonal of 5 x 5 matrix— (x + iy) ?

- are we encoding the parameters in A or b? <br />
The answer can be found by understanding the hough transform what is the step where linear equations are being solved in context of tracking.

- making A as hermitian can be done. It will be sparse, anyway. And Low-Rank. (How to ensure that?) (Wait, what was the requirements for matrix A, again? :| )

- Simulétor, I choose you! - quantumSim vs QuEST
